In [6]:
import pandas as pd
import requests
import time

In [7]:
# Riot Developer Portal에서 받은 API KEY입니다.
# 해당 값을 포함한 모든 종류의 KEY는 코드에 직접적으로 노출되지 않도록 하는 것이 좋습니다.
api_key = 'RGAPI-80e3307a-8620-40dc-bcef-b78a866b75aa'

# 소환사명입니다.
summoner_name = 'wincg'

In [8]:
# SUMMONER-V4 API URL
requesturl = 'https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/'+summoner_name+\
'?api_key='+api_key

In [9]:
# puuid입니다.
summoner_puuid = requests.get(requesturl).json()['puuid']

In [11]:
# 솔랭 게임 목록 가져오기

games_list = []
index = 0
while True:
    requesturl = 'https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/'+summoner_puuid+\
'/ids?queue=420&start='+str(index)+'&count=100&api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_list_new = response.json()
            break
    
    games_list.extend(game_list_new)
    
    if len(game_list_new) != 100:
        break
        
    index += 100
    time.sleep(1)

In [25]:
len(games_list)

144

In [23]:
# 솔랭 게임 각각에 대해서 API 호출 및 row binding
games_df = pd.DataFrame()
for matchId in games_list:
    requesturl = 'https://europe.api.riotgames.com/lol/match/v5/matches/'+matchId+\
    '?api_key='+api_key
    
    while True:
        response = requests.get(requesturl)
        if response.ok:
            game_json = response.json()
            break
    game_df = pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]
    game_df = game_df[game_df.puuid == summoner_puuid][['championId','win','kills','deaths','assists']]
        
    games_df = pd.concat([games_df,game_df])
    time.sleep(1)

In [26]:
pd.DataFrame(game_json['info']['participants'])[['puuid','championId','win','kills','deaths','assists']]

,puuid,championId,win,kills,deaths,assists
0,m8-eglJvXFOvVN8tY20zvX5uoQtz3NU6KyZbiNvAvxWE3A...,164,False,0,7,1
1,yUpXBeOyBaOV1lGNYucypWU_nJHjY0QS27C--xgClbsoUo...,64,False,3,10,3
2,p26UD2kopNx4sl4yzpf0qpHbJK7rVpGi574FKvaHAMTPd3...,517,False,2,12,4
3,5GQ29g0cUHOxA8xQ4uM5khRCmcysRCmckmfKI6PNR8xpOn...,81,False,6,4,0
4,8ti1BH6sZqqcOPs2ztI0ptY3WakxlF_GsCk-Z0Svo4oZ4q...,16,False,0,4,7
5,JzgsFeWtWS7UHM24wf1qMGYtOL1RnHJgJw92xVoV8D1ZtQ...,266,True,6,0,10
6,17rBdXDbj9HoUYRfMuSADy3doVi5yc3S8tu_BNmZjJzwky...,234,True,9,2,9
7,PGRXDQ66lfirmxzQMzq3WhwAeoFlbDk8dAtC1GeOGQONMI...,7,True,9,2,14
8,983JjZMcrCloY6C4XNL_7X69a4d4cPWe7soJMnSe1kbxMo...,18,True,8,4,10
9,NjCAIYFqkvzqQjXX9ucAjj36SZRh2KXvcX2-UVJOAWIklM...,111,True,5,3,18


In [27]:
games_df = games_df.groupby(['championId'],as_index=False).\
apply(lambda x: pd.Series({'win_count':sum(x.win),
                           'count':len(x),
                           'kills':sum(x.kills),
                           'deaths':sum(x.deaths),
                           'assists':sum(x.assists)
                          }))

In [28]:
# 챔피언 Key값에 대한 Value 가져오기
last_version = requests.get("https://ddragon.leagueoflegends.com/api/versions.json").json()[0]
champion_constant = requests.get("http://ddragon.leagueoflegends.com/cdn/"+last_version+"/data/ko_KR/champion.json")

In [29]:
# Json 파일을 DataFrame으로 변환
champion_df = pd.DataFrame(champion_constant.json()['data']).T[['key','name']]

# 변수형 문자 -> 숫자
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [30]:
games_df = games_df.merge(champion_df,
                          how = 'left',
                          left_on = 'championId',
                          right_on = 'key'
                         )[['name','count','win_count','kills','deaths','assists']]

In [31]:
games_df['WR'] = games_df['win_count'] / games_df['count']
games_df['KDA'] = (games_df['kills']+games_df['assists']) / games_df['deaths']

In [32]:
pd.set_option('display.max_columns', None)

In [36]:
games_df.sort_values('count',ascending = False)[['name','WR','count','KDA']]

,name,WR,count,KDA
2,르블랑,0.607143,28,4.704545
0,트위스티드 페이트,0.526316,19,3.475610
30,사일러스,0.789474,19,3.431579
12,리 신,0.909091,11,6.487179
22,조이,0.727273,11,4.757576
5,트린다미어,0.571429,7,3.320000
4,라이즈,0.600000,5,2.450000
16,탈론,1.000000,4,8.875000
6,신지드,0.750000,4,5.230769
31,요네,1.000000,3,2.363636


In [22]:
games_df.describe(include='all',datetime_is_numeric=True)

,name,count,win_count,kills,deaths,assists,WR,KDA
count,26,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000,26.000000
unique,26,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,오리아나,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.807692,2.500000,27.307692,14.961538,28.230769,0.578817,4.004942
std,NaN,4.931687,3.349627,42.317627,20.547468,40.557424,0.423432,4.084861
min,NaN,1.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.500000
25%,NaN,1.000000,0.000000,5.000000,5.000000,8.250000,0.000000,2.050000
50%,NaN,2.000000,1.000000,8.000000,8.000000,13.500000,0.708333,2.937500
75%,NaN,3.750000,3.000000,28.500000,12.750000,18.750000,1.000000,4.916667
